In [17]:
import multiprocessing as mp
import socket
import time

In [2]:
mp.get_all_start_methods(), mp.get_start_method()

(['spawn', 'fork', 'forkserver'], 'spawn')

In [3]:
def print_data():
    print("42")

proc = mp.Process(target=print_data)
proc.start()
proc.join()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
    from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=66, pipe_handle=82)
                                                  ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'print_data' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


In [18]:
def print_data():
    print("42")


proc_ctx = mp.get_context("fork")

proc = proc_ctx.Process(target=print_data)
proc.start()
proc.join()

42


In [10]:
mp.get_start_method()

'spawn'

In [11]:
class Person:
    def __init__(self, name):
        self.name = name


def update_data(data):
    print("update_data", data)

    data[1] = "one"
    data[("str", 42)] = "str42"
    data["pers"] = Person("steve")
    
    print("finished update_data", data)

In [14]:
update_data({})

update_data {}
finished update_data {1: 'one', ('str', 42): 'str42', 'pers': <__main__.Person object at 0x112cffed0>}


In [15]:
data = {1: "initial_one"}

print("data before update", data)

proc = proc_ctx.Process(target=update_data, args=(data,))
proc.start()
proc.join()

print("data after update", data)

data before update {1: 'initial_one'}
update_data {1: 'initial_one'}
finished update_data {1: 'one', ('str', 42): 'str42', 'pers': <__main__.Person object at 0x1131163f0>}
data after update {1: 'initial_one'}


In [23]:
class Person:
    def __init__(self, name):
        self.name = name


def update_data(data):
    print("update_data", data)

    data[1] = "one"
    data[("str", 42)] = "str42"
    data["pers"] = Person("steve")

    print("finished update_data", data)


with proc_ctx.Manager() as manager:
    data = manager.dict()
    data[1] = "initial_one"
    print("data before update", data)
    
    proc = proc_ctx.Process(target=update_data, args=(data,))
    proc.start()
    proc.join()

    print("data after update", data)
    backup = dict(data)

data before update {1: 'initial_one'}
update_data {1: 'initial_one'}
finished update_data {1: 'one', ('str', 42): 'str42', 'pers': <__main__.Person object at 0x1123d6cf0>}
data after update {1: 'one', ('str', 42): 'str42', 'pers': <__main__.Person object at 0x1123d6cf0>}


In [24]:
backup["pers"]

In [25]:
backup

{1: 'one', ('str', 42): 'str42', 'pers': <__main__.Person at 0x1123d4ec0>}

In [24]:
def server():
    print("start server")
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        sock.bind(("localhost", 20_000))
        sock.listen(5)

        i = 5
        while i > 0:
            i -= 1
            
            client_sock, address = sock.accept()
            print("server accepted client", address)

            data = client_sock.recv(1048)
            data = data.decode().upper()
            client_sock.sendall(data.encode())
            print("server finished to client", address)

    print("server finished")


def client(data):
    print("start client")
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect(("localhost", 20_000))

        sock.sendall(data.encode())

        data = sock.recv(1048).decode()
        print("client recv from server", data)

    print("client finished")


proc_server = proc_ctx.Process(target=server)
proc_client = proc_ctx.Process(target=client, args=("кружка",))

proc_server.start()
time.sleep(1)
proc_client.start()

# proc_server.join()
proc_client.join()

print("total finish")

start server
start client
server accepted client ('127.0.0.1', 50275)
server finished to clientclient recv from server  ('127.0.0.1', 50275)КРУЖКА

client finished
total finish
server accepted client ('127.0.0.1', 50316)
server finished to client ('127.0.0.1', 50316)
server accepted client ('127.0.0.1', 50350)
server finished to client ('127.0.0.1', 50350)
server accepted client ('127.0.0.1', 50374)
server finished to client ('127.0.0.1', 50374)
server accepted client ('127.0.0.1', 50412)
server finished to client ('127.0.0.1', 50412)
server finished


In [25]:
proc_server.is_alive()

True

In [26]:
proc_client = proc_ctx.Process(target=client, args=("ноут",))
proc_client.start()
proc_client.join()

start client
client recv from server НОУТ
client finished


In [27]:
proc_client = proc_ctx.Process(target=client, args=("ноут",))
proc_client.start()
proc_client.join()

start client
client recv from server НОУТ
client finished


In [28]:
proc_client = proc_ctx.Process(target=client, args=("ноут",))
proc_client.start()
proc_client.join()

start client
client recv from server НОУТ
client finished


In [29]:
proc_client = proc_ctx.Process(target=client, args=("тетрадь",))
proc_client.start()
proc_client.join()

start client
client recv from server ТЕТРАДЬ
client finished


In [30]:
print(proc_server.is_alive())

proc_client = proc_ctx.Process(target=client, args=("ноут",))
proc_client.start()
proc_client.join()

False
start client


Process ForkProcess-31:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_76265/2931819710.py", line 27, in client
    sock.connect(("localhost", 20_000))
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
ConnectionRefusedError: [Errno 61] Connection refused


In [33]:
import asyncio

In [35]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    await say_after(1, 'hello')
    await say_after(2, 'world')
    print(f"finished at {time.strftime('%X')}")


t1 = time.time()

await main()

t2 = time.time()
print("total_time", t2 - t1)

started at 21:04:06
hello
world
finished at 21:04:09
total_time 3.0034570693969727


In [36]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    say1 = say_after(1, 'hello')
    await say_after(2, 'world')
    await say1
    print(f"finished at {time.strftime('%X')}")


t1 = time.time()

await main()

t2 = time.time()
print("total_time", t2 - t1)

started at 21:04:47
world
hello
finished at 21:04:50
total_time 3.002131938934326


In [37]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    say1 = asyncio.create_task(say_after(1, 'hello'))
    await say_after(2, 'world')
    await say1
    print(f"finished at {time.strftime('%X')}")


t1 = time.time()

await main()

t2 = time.time()
print("total_time", t2 - t1)

started at 21:05:42
hello
world
finished at 21:05:44
total_time 2.001389980316162


In [ ]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    say1 = asyncio.create_task(say_after(1, 'hello'))
    await say_after(2, 'world')
    await say1
    print(f"finished at {time.strftime('%X')}")


t1 = time.time()

await main()

t2 = time.time()
print("total_time", t2 - t1)

In [38]:
async def say_after(delay, what):
    time.sleep(delay)
    print(what)


async def main():
    print(f"started at {time.strftime('%X')}")
    say1 = asyncio.create_task(say_after(1, 'hello'))
    await say_after(2, 'world')
    await say1
    print(f"finished at {time.strftime('%X')}")


t1 = time.time()

await main()

t2 = time.time()
print("total_time", t2 - t1)

started at 21:08:00
world
hello
finished at 21:08:03
total_time 3.0099399089813232


In [45]:
async def say_after(delay, what):
    t1 = time.time()
    await asyncio.sleep(delay)
    t2 = time.time()
    print(what, delay, t2 - t1)


async def main():
    print(f"started at {time.strftime('%X')}")
    
    say1 = asyncio.create_task(say_after(1, 'hello'))
    time.sleep(2)
    print("after sleep in main")
    say2 = asyncio.create_task(say_after(2, 'world'))
    
    await asyncio.gather(say1, say2)
    print(f"finished at {time.strftime('%X')}")


t1 = time.time()

await main()

t2 = time.time()
print("total_time", t2 - t1)

started at 21:17:17
after sleep in main
hello 1 1.000389814376831
world 2 2.0008420944213867
finished at 21:17:21
total_time 4.033520936965942


In [49]:
import aiohttp

In [66]:
URL = "https://docs.python.org/3/whatsnew/3.13.html"
URLS = [URL] * 10


async def fetch_url(session, url):
    await asyncio.sleep(1)
    # async with session.get(url) as resp:
    #     return await resp.text()


async def fetch_batch_urls(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_url(session, url) for url in urls]

        for task in tasks:
            await task


async def run():
    t1 = time.time()
    await fetch_batch_urls(URLS)
    t2 = time.time()
    print("batch time", t2 - t1)


await run()

batch time 10.082428216934204


In [67]:
URL = "https://docs.python.org/3/whatsnew/3.13.html"
URLS = [URL] * 10


async def fetch_url(session, url):
    await asyncio.sleep(1)
    # async with session.get(url) as resp:
    #     return await resp.text()


async def fetch_batch_urls(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_url(session, url) for url in urls]

        res = await asyncio.gather(*tasks)
        print(len(res))


async def run():
    t1 = time.time()
    await fetch_batch_urls(URLS)
    t2 = time.time()
    print("batch time", t2 - t1)


await run()

10
batch time 1.1302556991577148


In [70]:
URL = "https://docs.python.org/3/whatsnew/3.13.html"
URLS = [URL] * 10
WORKERS = 2


async def fetch_url(session, url):
    await asyncio.sleep(1)
    # async with session.get(url) as resp:
    #     return await resp.text()


async def fetch_worker(session, que, name):
    print("fetch_worker started", name)
    while True:
        url = await que.get()

        if url is None:
            await que.put(url)
            break

        try:
            result = await fetch_url(session, url)
        except Exception as err:
            print("error", err)

    print("fetch_worker finished", name)


async def run():
    t1 = time.time()

    que = asyncio.Queue()
    for url in URLS:
        await que.put(url)
    await que.put(None)

    async with aiohttp.ClientSession() as session:
        workers = [
            fetch_worker(session, que, f"worker_{i}")
            for i in range(WORKERS)
        ]
        await asyncio.gather(*workers)

    t2 = time.time()
    print("batch time", t2 - t1)


await run()

fetch_worker started worker_0
fetch_worker started worker_1
fetch_worker finished worker_0
fetch_worker finished worker_1
batch time 5.003886938095093


In [1]:
import asyncio
import aiohttp
import time

In [3]:
URL = "https://docs.python.org/3/whatsnew/3.13.html"
URLS = [URL] * 10
WORKERS = 2


async def fetch_url(session, url):
    await asyncio.sleep(1)
    # async with session.get(url) as resp:
    #     return await resp.text()


async def fetch_worker(session, que, name):
    print("fetch_worker started", name)
    while True:
        url = await que.get()

        try:
            result = await fetch_url(session, url)
        except Exception as err:
            print("error", err)
        finally:
            que.task_done()

    print("fetch_worker finished", name)


async def run():
    t1 = time.time()

    que = asyncio.Queue()
    for url in URLS:
        await que.put(url)

    print("queue")

    async with aiohttp.ClientSession() as session:
        workers = [
            asyncio.create_task(
                fetch_worker(session, que, f"worker_{i}")
            )
            for i in range(WORKERS)
        ]
        print("workers")

        await que.join()

        for worker in workers:
            worker.cancel()

    t2 = time.time()
    print("batch time", t2 - t1)


await run()

queue
workers
fetch_worker started worker_0
fetch_worker started worker_1
batch time 5.025568008422852


In [33]:
import time
import asyncio
import aiohttp

In [34]:
def do_timeit(coro):
    async def inner(*args, **kwargs):
        t1 = time.time()
        try:
            res = await coro(*args, **kwargs)
        finally:
            t2 = time.time()
            print(f"total time: {t2 - t1}")

        return res

    return inner

In [37]:
URL = "https://docs.python.org/3/whatsnew/3.13.html"
URLS = [URL] * 20
WORKERS = 10


async def fetch_url(session, sem, url):
    async with sem:
        await asyncio.sleep(1)
        # async with session.get(url) as resp:
        #     return await resp.text()


async def fetch_worker(session, que, sem, name):
    print("fetch_worker started", name)
    while True:
        url = await que.get()

        try:
            result = await fetch_url(session, sem, url)
        except Exception as err:
            print("error", err)
        finally:
            que.task_done()

    print("fetch_worker finished", name)


@do_timeit
async def run():
    que = asyncio.Queue()
    for url in URLS:
        await que.put(url)

    print("queue")

    sem = asyncio.Semaphore(3)

    async with aiohttp.ClientSession() as session:
        workers = [
            asyncio.create_task(
                fetch_worker(session, que, sem, f"worker_{i}")
            )
            for i in range(WORKERS)
        ]
        print("workers")

        await que.join()

        for worker in workers:
            worker.cancel()


await run()

queue
workers
fetch_worker started worker_0
fetch_worker started worker_1
fetch_worker started worker_2
fetch_worker started worker_3
fetch_worker started worker_4
fetch_worker started worker_5
fetch_worker started worker_6
fetch_worker started worker_7
fetch_worker started worker_8
fetch_worker started worker_9
total time: 7.023946046829224


In [39]:
async def bloking_api(n):
    print(f"start blocking api {n=}")

    time.sleep(n)

    print(f"finish blocking api {n=}")


async def non_blocking_api(n):
    print(f"start non-blocking api {n=}")

    await asyncio.sleep(n)
    print(f"finish non-blocking api {n=}")


@do_timeit
async def run(n):
    await asyncio.gather(bloking_api(n), non_blocking_api(n))


await run(5)

start blocking api n=5
finish blocking api n=5
start non-blocking api n=5
finish non-blocking api n=5
total time: 10.006846189498901


In [40]:
async def bloking_api(n):
    print(f"start blocking api {n=}")

    await asyncio.to_thread(time.sleep, n)

    print(f"finish blocking api {n=}")


async def non_blocking_api(n):
    print(f"start non-blocking api {n=}")

    await asyncio.sleep(n)
    print(f"finish non-blocking api {n=}")


@do_timeit
async def run(n):
    await asyncio.gather(bloking_api(n), non_blocking_api(n))


await run(5)

start blocking api n=5
start non-blocking api n=5
finish non-blocking api n=5
finish blocking api n=5
total time: 5.005144119262695


In [41]:
import concurrent.futures

In [42]:
concurrent.futures.Future

concurrent.futures._base.Future

In [43]:
import asyncio.futures

In [44]:
concurrent.futures.Future is asyncio.futures.Future

False

In [79]:
async def bloking_api(n):
    print(f"start blocking api {n=}")

    res = factorial(350_000)

    print(f"finish blocking api {n=}")


async def non_blocking_api(n):
    print(f"start non-blocking api {n=}")

    await asyncio.sleep(n)
    print(f"finish non-blocking api {n=}")


@do_timeit
async def run(n):
    await asyncio.gather(bloking_api(n), non_blocking_api(n))


await run(2)

start blocking api n=2
finish blocking api n=2
start non-blocking api n=2
finish non-blocking api n=2
total time: 3.9637069702148438


In [80]:
async def bloking_api(n):
    print(f"start blocking api {n=}")

    await asyncio.to_thread(factorial, 350_000)

    print(f"finish blocking api {n=}")


async def non_blocking_api(n):
    print(f"start non-blocking api {n=}")

    await asyncio.sleep(n)
    print(f"finish non-blocking api {n=}")


@do_timeit
async def run(n):
    await asyncio.gather(bloking_api(n), non_blocking_api(n))


await run(2)

start blocking api n=2
start non-blocking api n=2
finish blocking api n=2
finish non-blocking api n=2
total time: 2.316499948501587
